## Robo de cobrança de pedidos de fornecedores CDs PR e SP Varejo - Grupo Amigão

> Essa **automação** envia e-mails de cobrança de agendamentos por fornecedor para cada pedido **ativo** que constar sem agenda no relatório automatizado carteira de pedidos.

>Dica:
>
>Para executar esse código manualmente na totalidade, clique em **RUN** e depois em **RUN ALL CELLS**.

In [ ]:
# libs
import win32com.client as win32
import pandas as pd
from datetime import datetime, timedelta

In [ ]:
# bases
base = pd.read_excel('planilhas/base_dashboard.xlsx')
emails_forn = pd.read_excel('planilhas/emails_forn.xlsx')
emails_amigao = pd.read_excel('planilhas/emails_amigao.xlsx')

In [ ]:
data_atual = datetime.now().date()

In [ ]:
# filtra fornecedores com dt_entrega maior ou igual ao dial atual
base_dashboard = base[base['DT_ENTREGA'].dt.date >= data_atual + timedelta(days=3)] 

In [ ]:
# filtra fornecedores sem agendamento
fornecedores_sem_agendamento = base_dashboard[base_dashboard['DT_AGENDA'].str.startswith('SEM')]

In [ ]:
# filtra pedidos do tipo original e CD
fornecedores_cd_original = fornecedores_sem_agendamento[(fornecedores_sem_agendamento['Classificacao do pedido'] == 'Original') & 
                            (fornecedores_sem_agendamento['LOCAL_ENT'] == 'CD')]

In [ ]:
#fornecedores_cd_original = fornecedores_cd_original[fornecedores_cd_original['N_LJ'] != 745]

In [ ]:
# filtro sem CDs, pegar pedidos somente de lojas
fornecedores_cd_original = fornecedores_cd_original[(fornecedores_cd_original['N_LJ'] != 745) & (fornecedores_cd_original['N_LJ'] != 61)]

In [ ]:
fornecedores_validos = fornecedores_cd_original.drop_duplicates(subset="PEDIDO")

In [ ]:
# função de envio e-mail
def send_email(to, cc, subject, body):
    outlook = win32.Dispatch('outlook.application')
    mail = outlook.CreateItem(0)  # inicializa o outlook
    
    to_unique = list(set(to))
    cc_unique = list(set(cc))
    
    mail.To = ';'.join(to_unique)
    
    if any(cc_unique):
        mail.CC = ';'.join(cc_unique)

    mail.Subject = subject
    mail.Body = body
    
    try:
        mail.Send()
        return True 
    except Exception as e:
        print(f"Erro ao enviar e-mail: {str(e)}")
        return False 

# contador emails
emails_enviados = 0
emails_nao_enviados = 0

# agrupamento de fornecedores
for (usuario, depto, nome_fornecedor), grupo_pedidos in fornecedores_validos.groupby(['NOME USUARIO', 'DEP', 'DESC_FORN']):

    outlook = win32.Dispatch('outlook.application')
    mail = outlook.CreateItem(0)  # inicializa o outlook

    email_to = emails_forn.loc[emails_forn['nome_fornecedor'] == nome_fornecedor, 'email'].tolist()

    if not any(email_to):
        continue 

    email_cc = emails_forn.loc[emails_forn['nome_fornecedor'] == nome_fornecedor, 'email_forn_cc'].tolist()

    email_to = [str(email) for email in email_to]
    email_cc = [str(email) for email in email_cc]

    email_to = list(set(email_to))
    email_cc = list(set(email_cc))

    depto_fornecedor = grupo_pedidos.iloc[0]['DEP']
    
    if any(email_to):

        mail.To = ';'.join(email_to)
        
        if any(email_cc):
            mail.CC = ';'.join(email_cc)

        depto_fornecedor = grupo_pedidos.iloc[0]['DEP']
        if depto_fornecedor in emails_amigao['depto'].values:
            contatos_amigao = emails_amigao.loc[emails_amigao['depto'] == depto_fornecedor, 'contatos_amigao'].tolist()
            
            if any(contatos_amigao):
                email_cc += contatos_amigao

                email_cc = list(set(email_cc))

    # corpo do e-mail
    corpo_email = f"""Prezado Lorem Ipsum:

Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.

FORNECEDOR: {nome_fornecedor}
COD_FORN: {grupo_pedidos.iloc[0]['COD_FORN']}
DEPARTAMENTO: {depto_fornecedor}
COMPRADOR: {usuario}

PEDIDOS:\n"""
    for (_, pedido) in grupo_pedidos.groupby(['DEP', 'NOME USUARIO', 'DESC_FORN', 'PEDIDO']):
        lead_time = pedido.iloc[0]['DT_ENTREGA'].strftime('%d/%m/%y')  # formata datas para dd/mm/yy
        data_emissao = pedido.iloc[0]['DT_EMISSAO'].strftime('%d/%m/%y')  # formata datas para dd/mm/yy
        corpo_email += f"N° {pedido.iloc[0]['PEDIDO']} | EMITIDO EM: {data_emissao} | AGENDAR ATÉ: {lead_time}\n" 

    corpo_email += """


Quis autem vel eum iure reprehenderit qui in ea voluptate velit esse quam nihil molestiae consequatur, vel illum qui dolorem eum fugiat quo voluptas nulla pariatur?

Letraset sheets containing Lorem Ipsum passages, and more recently with desktop publishing.

Lorem Ipsum.
"""

    if send_email(email_to, email_cc, 'Lorem – Letraset sheets containing Lorem Ipsum passages - Lorem Ipsum', corpo_email):
        emails_enviados += 1
        print(f'E-mail enviado para {nome_fornecedor} ({usuario}, {depto})')
    else:
        emails_nao_enviados += 1

# log contadores envios
print(f'Total de e-mails enviados: {emails_enviados}')
print(f'Total de e-mails não enviados: {emails_nao_enviados}')